Для первой задачи мы используем данные [Jester Online Joke Recommender System](https://goldberg.berkeley.edu/jester-data/)

**Описание данных**

Файл `train_joke_df.csv` содержит:
- UID - id пользователей
- JID - id шуток, которые 
- Ratin - рейтинг шутки, который проставил пользователь 


Рейтинг имеет значение от -10.00 до 10.00. Могут встречаться значения 99.00, но это обозначает Null (нет рейтинга от пользователя).

Метрика для оценки [RMSE](https://www.codecamp.ru/blog/how-to-interpret-rmse/)

Минимальный RMSE: `4.2238`



In [1]:
#!pip install "scikit-surprise==1.1.3"

In [2]:
#!pip install "xlrd==2.0.1"

### Import

In [3]:
import numpy as np
import pandas as pd
from collections import defaultdict
from surprise import Dataset, Reader, KNNWithMeans, accuracy
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from sklearn.model_selection import train_test_split as tts
from surprise.model_selection import KFold
from tqdm.notebook import tqdm

np.random.seed(42)

In [4]:
n_clust = 300

print(n_clust)

300


### Базовые функции для скоринга и получения рекомендаций

In [5]:
def get_num_user_ratings(uid):
    """ возвращает кол-во рейтингов у пользователя 
    args: 
      uid: id пользователей
    returns: 
      кол-во объектов, которые оценил пользователь
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # пользователя не было во время обучения (новый, отправить на стартовые рекомендации)
        return 0
    
def get_num_item_ratings(iid):
    """ возвращает кол-во пользователей, которые оценили выбранный элемент 
    args:
      iid: строка с элементов рекомендации
    returns:
      кол-во пользователей, которые дали оценки по элементу
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
# На основе Surprise FAQ построим рекомендации Топ-N
def get_top_n(predictions, n=5):
    """Определят Топ-N рекомендаций

    Args:
        predictions(list of Prediction objects): Списко рекомендаций, из алгоритма Surprise
        n(int): Кол-во топ рекомендаций

    Returns:
        Словарь пользователь - список рекомендакиций для пользователей
        [(raw item id, rating estimation), ...]
    """

    # Предикт для каждого пользователя
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Сортировка предикта (по пользователям)
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n    

### Загрузка и обработка данных

In [6]:
df = pd.read_csv(r'..\data\recsys-in-practice\train_joke_df.csv')

df.head(5)

,UID,JID,Rating
0,18029,6,-1.26
1,3298,64,-4.17
2,3366,58,0.92
3,12735,92,3.69
4,11365,38,-6.60


In [7]:
cluster_df = pd.read_csv(f'cluster_df_{n_clust}.csv', index_col=1)
cluster_df

,cluster_id
UID,
1,52
2,249
3,95
4,194
5,70
...,...
24979,270
24980,222
24981,25


In [8]:
df = df.merge(cluster_df, left_on='UID', right_index=True)
df

,UID,JID,Rating,cluster_id
0,18029,6,-1.26,67
18990,18029,40,8.30,67
50486,18029,7,3.30,67
69643,18029,16,-9.27,67
124589,18029,15,8.11,67
...,...,...,...,...
1303561,21379,62,0.15,192
1331118,21379,46,1.31,192
1368576,21379,49,-0.68,192
1376073,21379,54,2.33,192


In [9]:
df.head(5)

,UID,JID,Rating,cluster_id
0,18029,6,-1.26,67
18990,18029,40,8.30,67
50486,18029,7,3.30,67
69643,18029,16,-9.27,67
124589,18029,15,8.11,67


In [10]:
# сделаем сортировку и перепишем index
df = df.sort_values(by=['cluster_id', 'JID'])
df = df.reset_index(drop=True)

In [11]:
# создадим на основе набора данных
# поднабор, который требуется для библиотеки Surprise

# указываем минимальный и максимальный рейтинги
reader = Reader(rating_scale=(-10, 10))

# передаём набор, указывая последовательность колонок: user (raw) ids, item (raw) ids, ratings
# для Surprise - это обязательно
data = Dataset.load_from_df(df[['cluster_id', 'JID', 'Rating']], reader)

In [12]:
trainset_data = data.build_full_trainset()

# сделаем разделение на обучающую и тестовую выборку
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

### Обучение модели

In [13]:
# определим набор данных для GridSearchCV
sim_options = {
    "name": ["msd", "cosine"], # способы оценки похожести (в GridSearch)
    "min_support": [3, 4],     # минимальное кол-во общих пользоватлей с данной шуткой
    "user_based": [False],     # поиск "похожести" будет на основе шуток, а не пользователей
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=2,)
gs.fit(data)
     
# результат
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
3.9960224103192297
{'sim_options': {'name': 'cosine', 'min_support': 3, 'user_based': False}}


In [14]:
# обучим с лучшими параметрами
algo = gs.best_estimator['rmse']
algo.fit(trainset)

# получим предикт и посмотрим метрику
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 3.9865


3.9865106653022955

### Тестирование и результаты

In [15]:
# получаем предикт
uid = 1  # id пользователя 
iid = 1  # iв шутки

# получим предик на основе обученных данных
# -7.82 - это фактический рейтинг, но посмотрим, какой ответ будет в предикте
pred = algo.predict(uid, iid, r_ui=-7.82, verbose=True)

user: 1          item: 1          r_ui = -7.82   est = 1.37   {'actual_k': 40, 'was_impossible': False}


In [16]:
uid = 24983  # id пользователя 
iid = 62     # iв шутки

pred = algo.predict(uid, iid, r_ui=-0.29, verbose=True)

user: 24983      item: 62         r_ui = -0.29   est = 0.88   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


### Обзор рекомендаций

In [17]:
# построим таблицу для обзора набора рекомендаций
# посмотрим, какие элементы и в каком кол-ве рекомендуем
trainset = algo.trainset

predictions_df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])

predictions_df['№ кол-во пользовательских рейтингов'] = predictions_df.uid.apply(get_num_user_ratings)
predictions_df['№ кол-во рейтингов элементов'] = predictions_df.iid.apply(get_num_item_ratings)
predictions_df['error'] = abs(predictions_df.est - predictions_df.rui)

best_predictions = predictions_df.sort_values(by='error')[:10]
worst_predictions = predictions_df.sort_values(by='error')[-10:]

In [18]:
best_predictions.head(5)

,uid,iid,rui,est,details,№ кол-во пользовательских рейтингов,№ кол-во рейтингов элементов,error
51962,33,45,0.97,0.97,"{'actual_k': 40, 'was_impossible': False}",14338,13718,0.0
91679,1,61,5.63,5.63,"{'actual_k': 40, 'was_impossible': False}",4372,15950,0.0
105154,153,59,-0.68,-0.68,"{'actual_k': 40, 'was_impossible': False}",18138,11698,0.0
54946,254,34,-3.54,-3.54,"{'actual_k': 40, 'was_impossible': False}",5933,13789,0.0
44167,274,18,-0.53,-0.53,"{'actual_k': 40, 'was_impossible': False}",7539,16129,0.0


In [19]:
# Предикт для всех, кого нет в выборке для обучения
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions)

# Сделаем вывод рекомендаций
a=0
for uid, user_ratings in top_n.items():
    a+=1
    print(uid, [iid for (iid, _) in user_ratings])
    
    if a==10:
        break

286 [100]
231 [89, 36, 35, 72, 93]
3 [96, 79]
240 [36, 32, 49, 54, 72]
72 [92, 85, 94, 82, 90]
210 [55, 75, 89, 72, 100]
220 [28, 80, 23, 4]
28 [87, 86, 72, 74]
188 [72, 35, 53, 29, 61]
78 [73]


### Для отправки на тестирование

In [20]:
test = pd.read_csv(r'..\data\recsys-in-practice\test_joke_df_nofactrating.csv', index_col=0)
test.head(5)

,UID,JID
InteractionID,,
0,11228,39
1,21724,85
2,16782,56
3,12105,42
4,14427,2


In [21]:
test = test.join(cluster_df, on='UID', lsuffix='_left', rsuffix='_right')
test

,UID,JID,cluster_id
InteractionID,,,
0,11228,39,155
1,21724,85,168
2,16782,56,212
3,12105,42,298
4,14427,2,76
...,...,...,...
362086,3085,66,75
362087,13765,31,213
362088,10341,29,40


In [22]:
test['Rating'] = test[['cluster_id', 'JID']].apply(lambda x: algo.predict(x[0], x[1], verbose=False).est,
                                                      axis = 1)
                                                      


In [23]:
# вид набора данных, который должен быть отправлен для тестирования
test['Rating'].to_frame().head(5)

,Rating
InteractionID,
0,4.59686
1,-5.34350
2,-1.80025
3,7.44025
4,7.27750


In [25]:
# формирование файла для отправки в Kaggle
test['Rating'].to_frame().to_csv(f'baseline_cluster_{n_clust}.csv')

In [26]:
algo.predict(11228, 39, verbose=False)

Prediction(uid=11228, iid=39, r_ui=None, est=0.880743062645692, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [27]:
algo.estimate(u=11228, i=39)

PredictionImpossible: User and/or item is unknown.

In [28]:
algo.sim.shape

(100, 100)